In [9]:
import json
import numpy as np

In [2]:
datafile = "reviews_Baby_5.json/datawithcomma"
jsonfile = open(datafile)
data = json.load(jsonfile)
userlist ={x['reviewerID']:[] for x in data}
# uniqreviewer = list(set(reviewerIds))

userlist1 = list(map(lambda item:userlist[item["reviewerID"]].append((item["asin"],item["unixReviewTime"])),data))


In [3]:
removelist = [k for k in userlist if len(userlist[k]) < 10]

i=0
itemset=set()
print(len(userlist))
for key in removelist:
    del userlist[key]
for k in userlist:    
    userlist[k].sort(key = lambda x:x[1])
#     print(userlist[k][:][0])
    itemset = itemset.union(set([a for (a,b) in userlist[k]]))
    
print(len(userlist))



19445
4382


In [15]:

encodedict = {val:idx for idx,val in enumerate(list(itemset),start=1)}

# print(encodedict)

# featurevector = [[encodedict[x] for (x,y) in useritems] for useritems in userlist.values()]
featurevector=[]
maxlen = 0
for userseq in userlist.values():
#     featurevector.append([])
    (featurevector.append([encodedict[x] for (x,y) in userseq]))
    if len(featurevector[-1]) > maxlen:
        maxlen = len(featurevector[-1])
        


In [24]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

# The Embedding layer takes at least two arguments:
# the number of possible words in the vocabulary, here 1000 (1 + maximum word index),
# and the dimensionality of the embeddings, here 32.
embedding_dim=32




In [18]:

train_data = keras.preprocessing.sequence.pad_sequences(featurevector,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=maxlen)


In [22]:
print(train_data[0])


[ 608 3290 4511 2484 1339 4464 4038 5214 4919 1967 6720    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]


In [25]:


model = keras.Sequential([
  layers.Embedding(len(itemset), embedding_dim, input_length=maxlen)
  
])

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 125, 32)           220256    
Total params: 220,256
Trainable params: 220,256
Non-trainable params: 0
_________________________________________________________________
